In [1]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    # Create a PdfReader object
    reader = PdfReader(pdf_path)

    # Initialize a string to hold all the extracted text
    extracted_text = ""

    # Loop through each page and extract text
    for page in reader.pages:
        extracted_text += page.extract_text()

    # Print the extracted text
    # with open('extracted_text.txt', 'w') as text_file:
        # text_file.write(extracted_text)

    return extracted_text

In [2]:
pdf_path = 'data/Finance_Project.pdf'
text = extract_text_from_pdf(pdf_path)

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_text(text)
print(f"Split document into {len(chunks)} chunks.")

Split document into 371 chunks.


In [4]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"trust_remote_code":True}) 

C:\Users\bhavy\AppData\Local\Temp\ipykernel_14312\3230921713.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"trust_remote_code":True})
e:\Hackathons\IEEE\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
import pdfplumber

# Extract text from the PDF with page numbers
pdf_path = "your_file.pdf"
documents = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        # Split text into smaller chunks if needed
        documents.append({"text": text, "metadata": {"page_number": i + 1}})

# Now, add these documents to the Chroma database with the page number in metadata
db.add_documents(documents)

FileNotFoundError: [Errno 2] No such file or directory: 'your_file.pdf'

In [12]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_texts(
    chunks, embeddings, persist_directory="./chroma"
)

In [6]:
def get_similar(query):
    result = db.similarity_search_with_relevance_scores(query=query, k=1)
    # return result
    return [(doc.page_content, doc.metadata.get('source', 'Unknown')) for doc, _ in result]

In [47]:
query = "What are the main points of the document?"
relevant_chunks = get_similar(query)
print(relevant_chunks)

[(Document(metadata={}, page_content='6. Comparative  Volatility  Analysis  across  Time  Zones ………………………………... 3 3 \n7. Recommendations  and Implications  .............................................................................. 35 \n8. Conclusion  ........................................................................................................................ 37 \n9. Bibliography .................................................................................................................... 39'), -0.0417235927438766)]


C:\Users\bhavy\AppData\Local\Temp\ipykernel_26576\1372054378.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='6. Comparative  Volatility  Analysis  across  Time  Zones ………………………………... 3 3 \n7. Recommendations  and Implications  .............................................................................. 35 \n8. Conclusion  ........................................................................................................................ 37 \n9. Bibliography .................................................................................................................... 39'), -0.0417235927438766)]
  result = db.similarity_search_with_relevance_scores(query=query, k=1)


In [7]:
PROMPT_TEMPLATE = """
Answer the user query using only the relevant information provided to you in this prompt.

Relevant context from documents:
{relevant}

user query: {query}
"""

In [8]:
import groq

In [9]:
messages = [{"role": "system", "content": "You are a helpful chatbot"}]

In [15]:
import os
from groq import Groq
from langchain.prompts import ChatPromptTemplate

def ask_ques(user_query):
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = template.format(
        relevant=get_similar(user_query),
        query=user_query
    )

    client = Groq(
        api_key="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",
    )

    messages.append({"role": "user", "content": prompt})

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
    )

    messages.pop()
    messages.append({"role": "user", "content": "user_query"})
    messages.append({"role": "assistant", "content": chat_completion.choices[0].message.content})

    print(f"Prompt: {prompt}")
    print(f"\n\nResponse: {chat_completion.choices}")

In [16]:
input = "Tell me about bitcoin volatility"
ask_ques(input)

Prompt: Human: 
Answer the user query using only the relevant information provided to you in this prompt.

Relevant context from documents:
[('helping the market mature and become a more viable and stable asset class for investors.  \n \nThe Volatility  of Bitcoin  and Its Role as a Medium  of Exchange  and a Store of Value  \n \nThe paper titled "The Volatility of Bitcoin and Its Role as a Medium of Exchange and a Store of Value" \noffers a thorough examination of Bitcoin\'s extreme price fluctuations and their implications for Bitcoin\'s', 'Unknown')]

user query: Tell me about bitcoin volatility



Response: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='According to the provided context, the paper titled "The Volatility of Bitcoin and Its Role as a Medium of Exchange and a Store of Value" offers a thorough examination of Bitcoin\'s extreme price fluctuations, which are commonly referred to as volatility.', role='assistant', function_cal

# Bakchodi

In [17]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"trust_remote_code":True})

In [97]:
query = "Your search query"
results = db.similarity_search_with_relevance_scores(query=query, k=5)

# Extract the text and metadata (including page number)
for result, score in results:
    context_text = result["text"]
    page_number = result["metadata"].get("page_number")
    print(f"Context: {context_text}")
    print(f"Page: {page_number}")

AttributeError: 'NoneType' object has no attribute 'similarity_search_with_relevance_scores'

In [18]:
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb

# Initialize the sentence transformer model for embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Chroma client
client = chromadb.Client()

# Create or load a Chroma collection
collection_name = "pdf_chunks"
collection = client.get_or_create_collection(name=collection_name)

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from each page of the PDF and returns it as a list of tuples,
    where each tuple contains the text of a page and its corresponding page number.
    """
    text_per_page = []
    
    # Open the PDF file
    reader = PdfReader(pdf_path)
    
    # Loop through all pages and extract text
    for page_num, page in enumerate(reader.pages, start=1):
        text = page.extract_text()  # Extract text from the page
        if text:  # Check if the page has any text
            text_per_page.append([text, page_num])  # Append tuple (text, page number)
    
    return text_per_page

def chunk_text(text, chunk_size=500):
    """
    Splits the text into smaller chunks of the specified size.
    """
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def add_text_chunks_to_chroma(pdf_path):
    """
    Extracts text from a PDF, breaks it into chunks, generates embeddings,
    and stores them in the Chroma database.
    """
    pdf_text = extract_text_from_pdf(pdf_path)
    
    chunks = []
    metadata = []
    ids = []
    
    for i, (text, page_num) in enumerate(pdf_text):
        # Chunk the text into smaller pieces
        chunked_text = chunk_text(text)
        chunks.extend(chunked_text)
        
        # Store metadata with page number for each chunk
        metadata.extend([{"page_num": page_num}] * len(chunked_text))
        
        # Generate unique IDs for each chunk
        ids.extend([f"chunk_{i}_{j}" for j in range(len(chunked_text))])
    
    # Generate embeddings for all chunks
    embeddings = embedder.encode(chunks, convert_to_tensor=False)
    
    # Add the chunks, metadata, embeddings, and IDs to the Chroma collection
    collection.add(
        ids=ids,
        documents=chunks,
        metadatas=metadata,
        embeddings=embeddings
    )

def search_in_chroma(query):
    """
    Performs a similarity search for the given query using the Chroma database.
    """
    # Generate embedding for the query
    query_embedding = embedder.encode(query, convert_to_tensor=False)
    
    # Perform similarity search in Chroma
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )
    
    # Return the top 5 results along with their metadata
    return results["documents"], results["metadatas"]

# Example usage
pdf_path = "data/Finance_Project.pdf"  # Replace this with the path to your PDF file
question = "Tell me something about bitcoin volatility"  # Replace this with your question

# Step 1: Add chunks of the PDF to Chroma
add_text_chunks_to_chroma(pdf_path)

# Step 2: Search for the question in the Chroma database
documents, metadatas = search_in_chroma(question)

# Step 3: Print out the results with page numbers
for doc, meta in zip(documents[0], metadatas[0]):
    print(f"Found in page {meta['page_num']}: {doc}")

Found in page 8: 8 must develop new tools and frameworks to better manage the risks associated with this emerging asset class. At the same time, regulators must work to provide clearer guidelines to ensure market stability and protect investors. Ultimately, a deeper understanding of cryptocurrency volatility will be essential in helping the market mature and become a more viable and stable asset class for investors. The Volatility of Bitcoin and Its Role as a Medium of Exchange and a Store of Value The paper titled "The Volatility of Bitcoin and Its Role as a Medium of Exchange and a Store of Value" offers a thorough examination of Bitcoin's extreme price fluctuations and their implications for Bitcoin's potential to function as a cur rency or store of value. Given Bitcoin's rising prominence in financial markets and its increasing adoption by both retail and institutional investors, the study aims to uncover how Bitcoin's volatility affects its utility in the world of finance. Bitcoin

In [23]:
def ask_for_questions():
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    
    # Retrieve top 5 relevant chunks
    relevant_chunks = get_similar("Generate questions from this document")

    # Format the prompt with the relevant information
    prompt = template.format(
        relevant=relevant_chunks,
        query="Generate questions from this document"
    )

    # Call the LLM API to generate questions
    client = Groq(
        api_key="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",
    )

    messages.append({"role": "user", "content": prompt})

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.1-8b-instant",
    )

    # Get and print the generated questions
    response = chat_completion.choices[0].message.content
    print(f"Generated Questions:\n{response}")

In [24]:
ask_for_questions()

C:\Users\bhavy\AppData\Local\Temp\ipykernel_14312\3116442797.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='5. Parth Kushwaha  : 23103064  2  Table  of Contents  \n1. Abstract  ...............................................................................................................................3  \n2. Introduction  ....................................................................................................................... 4 \n3. Review  of Literature  ......................................................................................................... 5 \n○ Need  of the Study'), -0.15770025590306203)]
  result = db.similarity_search_with_relevance_scores(query=query, k=1)


Generated Questions:
Here are some questions that can be generated from the provided document:

1. What is the main subject of the study mentioned in the table of contents?
2. How many chapters does the study have, according to the table of contents?
3. What is the title of Abstract chapter in the study?
4. What is the number of the chapter in the Introduction section?
5. What is the chapter that reviews the Literature in the study?


In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from each page of the PDF and returns it as a list of tuples,
    where each tuple contains the text of a page and its corresponding page number.
    """
    text_per_page = []
    
    # Open the PDF file
    reader = PdfReader(pdf_path)
    
    # Loop through all pages and extract text
    for page_num, page in enumerate(reader.pages, start=1):
        text = page.extract_text()  # Extract text from the page
        if text:  # Check if the page has any text
            text_per_page.append((text, page_num))  # Append tuple (text, page number)
    
    return text_per_page

In [2]:
import fitz  # PyMuPDF
from pypdf import PdfReader
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import io
import groq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import os
from groq import Groq
from langchain.prompts import ChatPromptTemplate
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb

# Initialize the sentence transformer model for embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Chroma client
client = chromadb.Client()

# Create or load a Chroma collection
collection_name = "pdf_chunks"
collection = client.get_or_create_collection(name=collection_name)

PROMPT_TEMPLATE = """
Answer the user query using only the relevant information provided to you in this prompt. Write questions inside <> wherever you think you might need more information and it will be provided to you. If the 'Your Response:' part is empty then you have to generate the questions, if it is not empty then you have to replace the questions with new information if it is of any use, otherwise just remove those questions

Your Response:
{your_response}

Relevant context from documents:
{relevant}

user query: {query}
"""

messages = [{"role": "system", "content": "You are a helpful chatbot"}]

def get_stuff(text, image_text, user_query, messages):
    # Step 1: Add chunks of the PDF to Chroma
    add_text_chunks_to_chroma(text, image_text)

    # Step 2: Search for the question in the Chroma database
    documents, metadatas = search_in_chroma(user_query)

    response1 = ask_ques(documents, user_query, messages)
    print(response1)

    # Step 3: Print out the results with page numbers
    # for doc, meta in zip(documents[0], metadatas[0]):
    #     print(f"Found in page {meta['page_num']}: {doc}")

def chunk_text(text, chunk_size=500):
    """
    Splits the text into smaller chunks of the specified size.
    """
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def add_text_chunks_to_chroma(text, image_text):
    """
    Extracts text from a PDF, breaks it into chunks, generates embeddings,
    and stores them in the Chroma database.
    """
    pdf_text = extract_text_from_pdf("data/Finance_Project.pdf")
    
    chunks = []
    metadata = []
    ids = []
    
    for i, (text, page_num) in enumerate(pdf_text):
        # Chunk the text into smaller pieces
        chunked_text = chunk_text(text)
        chunks.extend(chunked_text)
        
        # Store metadata with page number for each chunk
        metadata.extend([{"page_num": page_num}] * len(chunked_text))
        
        # Generate unique IDs for each chunk
        ids.extend([f"chunk_{i}_{j}" for j in range(len(chunked_text))])
    
    chunks = chunks + image_text

    # Generate embeddings for all chunks
    embeddings = embedder.encode(chunks, convert_to_tensor=False)
    
    # Add the chunks, metadata, embeddings, and IDs to the Chroma collection
    collection.add(
        ids=ids,
        documents=chunks,
        metadatas=metadata,
        embeddings=embeddings
    )

def search_in_chroma(query):
    """
    Performs a similarity search for the given query using the Chroma database.
    """
    # Generate embedding for the query
    query_embedding = embedder.encode(query, convert_to_tensor=False)
    
    # Perform similarity search in Chroma
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )
    
    # Return the top 5 results along with their metadata
    # return results["documents"], results["metadatas"]
    return results["documents"], results["metadatas"]

def ask_ques(results, user_query, messages):
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = template.format(
        relevant=results,
        query=user_query
    )

    client = Groq(
        api_key="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",
    )

    messages.append({"role": "user", "content": prompt})

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
    )

    messages.pop()
    messages.append({"role": "user", "content": user_query})
    messages.append({"role": "assistant", "content": chat_completion.choices[0].message.content})

    return chat_completion.choices[0].message.content

In [4]:
image_text = ['Image 1.1: education cart wait but why',
 'Image 2.1: the world is not a water - free environment',
 'Image 3.1: children education a little more thane',
 'Image 4.1: a woman is writing on a piece of paper',
 'Image 5.1: a child sitting on the floor with a book and pencil',
 'Image 6.1: a piece of paper with a hand holding a pen',
 'Image 7.1: classroom culture in order to achieve learning on time and movement',
 'Image 7.2: a col of images of children in a classroom',
 'Image 8.1: a col of images of children in a classroom',
 'Image 9.1: mental welling assessment - mental welling assessment',
 'Image 10.1: a col of photos of children in different places',
 'Image 11.1: impact road we believe that a metaphor is not an answer',
 'Image 12.1: two young boys writing in front of a book',
 'Image 13.1: a graph shows the percentage of children who have been diagnosed',
 "Image 14.1: education can't wait for the future"]

In [5]:
print(get_stuff(text, image_text, "Tell me something about bitcoin volatility", messages))

NameError: name 'text' is not defined

In [6]:
PROMPT_TEMPLATE = """
Answer the user query using only the relevant information provided to you in this prompt. 
Write questions inside <> wherever you think you might need more information and it will be provided to you.
If the 'Your Response:' part is empty then you have to generate the questions, if it is not empty then you have to replace the questions with new information if it is of any use, otherwise just remove those questions.

Your Response:
{your_response}

Relevant context from documents:
{relevant}

user query: {query}
"""

def ask_ques(results, user_query, messages, previous_response=""):
    """
    This function generates a response and potential clarifying questions.
    If previous_response is empty, it generates questions. Otherwise, it integrates new context.
    """
    template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    
    # Format prompt with the previous response and relevant context
    prompt = template.format(
        relevant=results,  # Pass relevant chunks or results from Chroma
        query=user_query,
        your_response=previous_response  # If empty, questions will be generated
    )

    client = Groq(
        api_key="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",
    )

    messages.append({"role": "user", "content": prompt})

    # Generate chat completion with Groq API
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
    )

    # Get the model's response (which may include questions)
    response_content = chat_completion.choices[0].message.content

    # Remove the last user prompt and update messages with new context
    messages.pop()
    messages.append({"role": "user", "content": user_query})
    messages.append({"role": "assistant", "content": response_content})

    return response_content

def extract_questions(response_content):
    """
    Extract questions from the model's response. Questions are inside <>.
    """
    questions = []
    start = response_content.find('<')
    
    while start != -1:
        end = response_content.find('>', start)
        if end != -1:
            question = response_content[start+1:end]
            questions.append(question)
            start = response_content.find('<', end)
        else:
            break
    
    return questions

def process_questions_and_search(questions):
    """
    Uses the questions to perform a similarity search in Chroma.
    """
    search_results = []
    for question in questions:
        # Perform vector similarity search using the extracted questions
        documents, metadatas = search_in_chroma(question)
        search_results.append((documents, metadatas))
    
    # Combine all search results into a single list
    combined_documents = [doc for docs, _ in search_results for doc in docs]
    return combined_documents

def generate_final_response(relevant_context, user_query, previous_response, messages):
    """
    Generate the final response by providing the previous response and new context from similarity search.
    """
    final_response = ask_ques(relevant_context, user_query, messages, previous_response)
    return final_response

# Main flow function
def get_stuff(text, image_text, user_query, messages):
    # Step 1: Add chunks of the PDF to Chroma
    add_text_chunks_to_chroma(text, image_text)

    # Step 2: Search for the initial query in the Chroma database
    documents, metadatas = search_in_chroma(user_query)
    
    # Step 3: Generate the initial model response with potential questions
    initial_response = ask_ques(documents, user_query, messages)
    
    # Step 4: Extract the questions from the model's response
    questions = extract_questions(initial_response)
    
    if questions:
        # Step 5: Use the questions to perform similarity search
        new_context = process_questions_and_search(questions)
        
        # Step 6: Generate the final response using the new context and the initial response
        final_output = generate_final_response(new_context, user_query, initial_response, messages)
        
        return final_output
    else:
        # If no questions were generated, return the initial response
        return initial_response